In [2]:
import os
import pandas as pd 

In [7]:
import os
import pandas as pd
from collections import defaultdict

input_folder = "./"

# Collect all CSV files in the folder
csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

# Track which file contains which columns
file_columns = {}

for file in csv_files:
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path, delimiter=",", low_memory=False)
    file_columns[file] = set(df.columns)

# Find overlapping column names (excluding "Start date")
overlap_report = defaultdict(list)

for i in range(len(csv_files)):
    file_i = csv_files[i]
    for j in range(i + 1, len(csv_files)):
        file_j = csv_files[j]
        common_cols = file_columns[file_i].intersection(file_columns[file_j])
        common_cols.discard("Start date")  # ignore merge key
        if common_cols:
            for col in common_cols:
                overlap_report[col].append((file_i, file_j))

# Print results
print("\n🔍 Duplicate Columns Found Across Files:\n")
if not overlap_report:
    print("No duplicate column names found across files.")
else:
    for col, file_pairs in overlap_report.items():
        print(f"Column: '{col}' found in:")
        for pair in file_pairs:
            print(f"  - {pair[0]} & {pair[1]}")
        print("-" * 40)



🔍 Duplicate Columns Found Across Files:

Column: 'Austria [MWh] Original resolutions' found in:
  - cleaned_Imported_balancing_services_202301010000_202503050000_Quarterhour.csv & cleaned_Exported_balancing_services_202301010000_202503050000_Quarterhour.csv
----------------------------------------
Column: 'Hydro pumped storage [MWh] Original resolutions' found in:
  - cleaned_Actual_generation_Quarterhour.csv & cleaned_Actual_consumption_Quarterhour.csv
----------------------------------------


In [ ]:
import os
import pandas as pd

input_folder = "./"

csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]
df_list = []

for file in csv_files:
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path, delimiter=",", low_memory=False)
    
    # Use filename (without extension) as suffix
    tag = os.path.splitext(file)[0].replace("cleaned_", "")
    
    # Rename all columns except "Start date"
    new_cols = []
    for col in df.columns:
        if col == "Start date":
            new_cols.append(col)
        else:
            new_cols.append(f"{col}_{tag}")
    df.columns = new_cols

    df_list.append(df)

# Merge all dataframes on 'Start date'
merged_df = df_list[0]
for df in df_list[1:]:
    merged_df = pd.merge(merged_df, df, on="Start date", how="inner")

print("All files merged and duplicate columns renamed based on source file.")
print("Final shape:", merged_df.shape)

# Optional: save result
merged_df.to_csv("./cleaned_and_merged.csv", index=False)


✅ All files merged and duplicate columns renamed based on source file.
📐 Final shape: (76525, 39)
